In [1]:
%load_ext autoreload 
%autoreload 2

In [2]:
%matplotlib inline

In [3]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "4" 

# 设置 HTTP 和 HTTPS 代理
os.environ["http_proxy"] = "***"
os.environ["https_proxy"] = "***"

# predict

## visual prompt
python predict_visual_prompt.py

In [4]:
import argparse
import os
from PIL import Image
import supervision as sv
from ultralytics import YOLOE
import numpy as np
from ultralytics.models.yolo.yoloe.predict_vp import YOLOEVPSegPredictor

In [5]:
import matplotlib.pyplot as plt

In [20]:
checkpoint = "./pretrain/yoloe-v8l-seg.pt"  # yoloe-v8s-seg-det
# checkpoint = "./pretrain/yoloe-v8s-seg.pt"
device = "cuda:0"
model = YOLOE(checkpoint)
model.to(device)

In [7]:
import cv2

source_image_path = './ultralytics/assets/bus.jpg'
target_image_path = './ultralytics/assets/zidane.jpg'

output = "/root/project/research/Yolo/yoloe/test_data_out/"
os.makedirs(output, exist_ok=True)

source_image = Image.open(source_image_path).convert("RGB")
target_image = Image.open(target_image_path).convert("RGB")

# visuals = dict(
#     bboxes=[
#         np.array([[221.52, 405.8, 344.98, 857.54], [120, 425, 160, 445]]), 
#         np.array([[150, 200, 1150, 700]])
#     ],
#     cls=[
#         np.array([0, 1]), 
#         np.array([0])
#     ]
# )

visuals = dict(
    bboxes=[
        np.array([[221.52, 405.8, 344.98, 857.54], [120, 425, 160, 445]])
    ],
    cls=[
        np.array([0, 1])
    ]
)

box1 = visuals['bboxes'][0][0]
box2 = visuals['bboxes'][0][1]
print(f"box1: {box1}, box2: {box2}")

# 转成整数坐标
pt11 = tuple(box1[:2].astype(int))
pt12 = tuple(box1[2:].astype(int))

pt21 = tuple(box2[:2].astype(int))
pt22 = tuple(box2[2:].astype(int))

# 绘制矩形
d_source_image = cv2.cvtColor(np.array(source_image), cv2.COLOR_RGB2BGR)
d_source_image = cv2.rectangle(d_source_image, pt11, pt12, (255, 0, 0), 5)
d_source_image = cv2.rectangle(d_source_image, pt21, pt22, (255, 0, 0), 5)
d_source_image = cv2.cvtColor(d_source_image, cv2.COLOR_BGR2RGB)

# fig, ax = plt.subplots(figsize=(8, 8))
# ax.imshow(d_source_image, cmap="gray") 
# fig, ax = plt.subplots(figsize=(8, 8))
# ax.imshow(target_image, cmap="gray") 

box1: [     221.52       405.8      344.98      857.54], box2: [        120         425         160         445]


In [8]:
visuals

{'bboxes': [array([[     221.52,       405.8,      344.98,      857.54],
         [        120,         425,         160,         445]])],
 'cls': [array([0, 1])]}

In [9]:
# results = model.predict([source_image, target_image], save=True, prompts=visuals, predictor=YOLOEVPSegPredictor)
results = model.predict([source_image], save=True, prompts=visuals, predictor=YOLOEVPSegPredictor)
# results = model.predict(source_image, save=False, prompts=visuals, predictor=YOLOEVPSegPredictor)


0: 640x640 4 object0s, 2 object1s, 444.0ms
Speed: 44.6ms preprocess, 444.0ms inference, 666.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to /root/project/research/Yolo/ultralytics/runs/segment/predict23


In [10]:
detections = sv.Detections.from_ultralytics(results[0])
# print(f"detections: {detections}")

resolution_wh = source_image.size

thickness = sv.calculate_optimal_line_thickness(resolution_wh=resolution_wh)
text_scale = sv.calculate_optimal_text_scale(resolution_wh=resolution_wh)

labels = [
    f"{class_name} {confidence:.2f}"
    for class_name, confidence in zip(detections["class_name"], detections.confidence)
]

annotated_image = source_image.copy()

annotated_image = sv.MaskAnnotator(
    color_lookup=sv.ColorLookup.INDEX,
    opacity=0.4
).annotate(scene=annotated_image, detections=detections)

annotated_image = sv.BoxAnnotator(
    color_lookup=sv.ColorLookup.INDEX,
    thickness=thickness
).annotate(scene=annotated_image, detections=detections)

annotated_image = sv.LabelAnnotator(
    color_lookup=sv.ColorLookup.INDEX,
    text_scale=text_scale,
    smart_position=True
).annotate(scene=annotated_image, detections=detections, labels=labels)

print(f"annotated_image: {annotated_image.size}")
# fig, ax = plt.subplots(figsize=(8, 8))
# ax.imshow(annotated_image, cmap="gray")

base, ext = os.path.splitext(source_image_path)
path, filename = os.path.split(source_image_path)
print(f"base: {path}, ext: {filename}")
output_name = os.path.join(output, "re_" + filename)
print(f"output_name: {output_name}")
annotated_image.save(output_name)

annotated_image: (810, 1080)
base: ./ultralytics/assets, ext: bus.jpg
output_name: /root/project/research/Yolo/yoloe/test_data_out/re_bus.jpg


## 一张图片的 visual prompt 去识别所有图片的对应目标

In [11]:
import argparse
import os
from PIL import Image
import supervision as sv
from ultralytics import YOLOE
import numpy as np
from ultralytics.models.yolo.yoloe.predict_vp import YOLOEVPSegPredictor

In [12]:
import matplotlib.pyplot as plt

In [19]:
checkpoint = "./pretrain/yoloe-v8l-seg.pt"
device = "cuda:0"
model = YOLOE(checkpoint)
model.to(device)

In [14]:
import cv2

# source_image_path = './test_data/000000039769.jpg'
source_image_path = "/root/dataset/glass_data_20250317/images/train/Image_20250210134520105.bmp"

output = "/root/project/research/Yolo/yoloe/test_data_out/"
os.makedirs(output, exist_ok=True)

source_image = Image.open(source_image_path).convert("RGB")

# visuals = dict(
#     bboxes=[
#         np.array([[221.52, 405.8, 344.98, 857.54], [120, 425, 160, 445]]), 
#         np.array([[150, 200, 1150, 700]])
#     ],
#     cls=[
#         np.array([0, 1]), 
#         np.array([0])
#     ]
# )

# visuals = dict(
#     bboxes=[
#         np.array([[221.52, 405.8, 344.98, 857.54], [120, 425, 160, 445]])
#     ],
#     cls=[
#         np.array([0, 1])
#     ]
# )

# ----------- green leaf
# visuals = dict(
#     bboxes=[
#         np.array([[662, 140, 1148, 532], [253, 609, 635, 788]])
#     ],
#     cls=[
#         np.array([0, 0])
#     ]
# )

# -------- cats -----------
# visuals = dict(
#     bboxes=[
#         np.array([[13, 56, 314, 470]])
#     ],
#     cls=[
#         np.array([0])
#     ]
# )

visuals = dict(
    bboxes=[
        np.array([[900, 650, 1200, 820]])
    ],
    cls=[
        np.array([0])
    ]
)

# 绘制矩形
d_source_image = cv2.cvtColor(np.array(source_image), cv2.COLOR_RGB2BGR)

# 遍历 bbox 并画框
for bbox in visuals["bboxes"][0]:
    pt1 = tuple(bbox[:2].astype(int))
    pt2 = tuple(bbox[2:].astype(int))
    d_source_image = cv2.rectangle(d_source_image, pt1, pt2, (255, 0, 0), 5)

d_source_image = cv2.cvtColor(d_source_image, cv2.COLOR_BGR2RGB)
print(f"d_source_image: {d_source_image.shape}")

# fig, ax = plt.subplots(figsize=(8, 8))
# ax.imshow(d_source_image, cmap="gray") 

d_source_image: (2048, 3072, 3)


In [15]:
# return_vpe=True 表示提取 VPE（Visual Prompt Embedding）用于后续使用。
model.predict(source_image, prompts=visuals, predictor=YOLOEVPSegPredictor, return_vpe=True)
# 相当于使用第一张图片的 visual prompt 去识别所有图片的对应目标
# model.set_classes(["person", "glasses"], model.predictor.vpe)
# model.set_classes(["green leaf"], model.predictor.vpe)
model.set_classes(["cat"], model.predictor.vpe)
model.predictor = None  # remove VPPredictor


0: 640x640 2 object0s, 42.4ms
Speed: 5.1ms preprocess, 42.4ms inference, 12.5ms postprocess per image at shape (1, 3, 640, 640)


In [16]:
import cv2
# target_image_path = './ultralytics/assets/zidane.jpg'
# target_image_path = './test_data/green_leaf.jpg'
# target_image_path = './test_data/000000039769.jpg'
target_image_path = "/root/dataset/glass_data_20250317/images/train/Image_20250210134520105.bmp"

target_image = Image.open(target_image_path).convert("RGB")

# results = model.predict(target_image, save=True)
results = model.predict(target_image, conf=0.25)


0: 640x640 2 cats, 16.6ms
Speed: 6.6ms preprocess, 16.6ms inference, 12.5ms postprocess per image at shape (1, 3, 640, 640)


In [17]:
detections = sv.Detections.from_ultralytics(results[0])
print(f"detections: {detections}")

resolution_wh = target_image.size

thickness = sv.calculate_optimal_line_thickness(resolution_wh=resolution_wh)
text_scale = sv.calculate_optimal_text_scale(resolution_wh=resolution_wh)

labels = [
    f"{class_name} {confidence:.2f}"
    for class_name, confidence in zip(detections["class_name"], detections.confidence)
]

annotated_image = target_image.copy()

annotated_image = sv.MaskAnnotator(
    color_lookup=sv.ColorLookup.INDEX,
    opacity=0.4
).annotate(scene=annotated_image, detections=detections)

annotated_image = sv.BoxAnnotator(
    color_lookup=sv.ColorLookup.INDEX,
    thickness=thickness
).annotate(scene=annotated_image, detections=detections)

annotated_image = sv.LabelAnnotator(
    color_lookup=sv.ColorLookup.INDEX,
    text_scale=text_scale,
    smart_position=True
).annotate(scene=annotated_image, detections=detections, labels=labels)

print(f"annotated_image: {annotated_image.size}")
# fig, ax = plt.subplots(figsize=(8, 8))
# ax.imshow(annotated_image, cmap="gray")

base, ext = os.path.splitext(target_image_path)
path, filename = os.path.split(target_image_path)
print(f"base: {path}, ext: {filename}")
output_name = os.path.join(output, "re_" + filename)
print(f"output_name: {output_name}")
annotated_image.save(output_name)

detections: Detections(xyxy=array([[     928.08,      663.96,      1180.9,      806.42],
       [     1428.9,      669.68,      1683.1,      811.23]], dtype=float32), mask=array([[[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]],

       [[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]]]), confidence=array([    0.98826,     0.97257], dtype=float32), class_id=array([0, 0]), tracker_id=None, data={'clas

In [18]:
# 所有目标的 mask
masks = results[0].masks.data.cpu().numpy()  # shape: (num_objects, H, W)
print("Mask shape:", masks.shape)

# 遍历每个目标的二值图
for i, mask in enumerate(masks):
    binary_mask = (mask > 0.5).astype(np.uint8) * 255  # 转为 uint8 的二值图
    print(f"binary_mask: {binary_mask.shape}")
    # fig, ax = plt.subplots(figsize=(8, 8))
    # ax.imshow(binary_mask, cmap="gray") 

Mask shape: (2, 640, 640)
binary_mask: (640, 640)
binary_mask: (640, 640)
